In [1]:
#엔진은 항상필요하다 
from sqlalchemy import create_engine

In [2]:
engine = create_engine("sqlite://", echo = True)

In [3]:
#metadata 같은것
from sqlalchemy.ext.declarative import declarative_base

In [4]:
Base = declarative_base() #metadata 와 비슷한 역할을 하는 베이스를 만든다

## 아까는 메타데이터를 상속받아서 테이블 지금은 베이스를 만들어서 클래스를 만든다.

In [5]:
from sqlalchemy import Column,Integer,String,ForeignKey

In [6]:
from sqlalchemy.orm import relationship #이게 달라진부분.

In [7]:
class Artist(Base):
    __tablename__ = "artist"
    
    id = Column(Integer, primary_key=True)
    name = Column(String)
    album = relationship("Album", back_populates = "artist")
    
    def __repr__(self):
        return "<T'artist(name='%s')>" % (self.name)
    
class Album(Base):
    __tablename__ = "album"
    
    id = Column(Integer, primary_key=True)
    title = Column(String)
    artist_id = Column(Integer, ForeignKey("artist.id"))
    artist = relationship("Artist", back_populates = "album") # uselist = false 로 주면 1대n 의 관계, list 타입이아니기때문 기본은 m:n이다 . 
    #PK,FK, 어떤 relationship 이 있는지 표현만 해주면 engine 과 base 가 알아서 해준다. 

class Genre(Base):
    __tablename__ = "genre"
    
    id = Column(Integer, primary_key=True)
    name = Column(String)
    
class Track(Base):
    __tablename__ = "track"
    
    id = Column(Integer, primary_key=True)
    title = Column(String)
    album_id = Column(Integer, ForeignKey("album.id"))
    genre_id = Column(Integer, ForeignKey("genre.id"))

In [8]:
Base.metadata.create_all(engine) # metadata 가 안에있다. 그래서 아까 배운거. 아까 만든 엔진을  binding 해준다
#클래스를 선언해서 만들어준거 밖에없는데 자기가 알아서 테이블을 만든다

2018-07-12 12:05:04,803 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2018-07-12 12:05:04,806 INFO sqlalchemy.engine.base.Engine ()
2018-07-12 12:05:04,808 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2018-07-12 12:05:04,808 INFO sqlalchemy.engine.base.Engine ()
2018-07-12 12:05:04,810 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("artist")
2018-07-12 12:05:04,811 INFO sqlalchemy.engine.base.Engine ()
2018-07-12 12:05:04,812 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("album")
2018-07-12 12:05:04,813 INFO sqlalchemy.engine.base.Engine ()
2018-07-12 12:05:04,813 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("genre")
2018-07-12 12:05:04,814 INFO sqlalchemy.engine.base.Engine ()
2018-07-12 12:05:04,815 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("track")
2018-07-12 12:05:04,816 INFO sqlalchemy.engine.base.Engine ()
2018-07-12 12:05:04,817 INFO sqlalchemy.engin

In [9]:
artist1 = Artist(name = 'Led zeppelin')
artist2 = Artist(name = 'AC/DC')

In [10]:
artist1.album = [Album(title="IV"), Album(title = "Who Made Who")] #이게가능해진

In [11]:
from sqlalchemy.orm import sessionmaker #session 을 만들어야한다

Session = sessionmaker(bind =engine) #엔진과 비인딩을 해준다

session = Session() #이때 세션을 생성



In [12]:
session.add(artist1)

In [13]:
session.commit()

2018-07-12 12:05:06,798 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-07-12 12:05:06,800 INFO sqlalchemy.engine.base.Engine INSERT INTO artist (name) VALUES (?)
2018-07-12 12:05:06,801 INFO sqlalchemy.engine.base.Engine ('Led zeppelin',)
2018-07-12 12:05:06,803 INFO sqlalchemy.engine.base.Engine INSERT INTO album (title, artist_id) VALUES (?, ?)
2018-07-12 12:05:06,804 INFO sqlalchemy.engine.base.Engine ('IV', 1)
2018-07-12 12:05:06,805 INFO sqlalchemy.engine.base.Engine INSERT INTO album (title, artist_id) VALUES (?, ?)
2018-07-12 12:05:06,806 INFO sqlalchemy.engine.base.Engine ('Who Made Who', 1)
2018-07-12 12:05:06,807 INFO sqlalchemy.engine.base.Engine COMMIT


In [14]:
print(artist1.id,artist1.name,artist1.album[0].id)

2018-07-12 12:05:07,294 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-07-12 12:05:07,295 INFO sqlalchemy.engine.base.Engine SELECT artist.id AS artist_id, artist.name AS artist_name 
FROM artist 
WHERE artist.id = ?
2018-07-12 12:05:07,296 INFO sqlalchemy.engine.base.Engine (1,)
2018-07-12 12:05:07,299 INFO sqlalchemy.engine.base.Engine SELECT album.id AS album_id, album.title AS album_title, album.artist_id AS album_artist_id 
FROM album 
WHERE ? = album.artist_id
2018-07-12 12:05:07,300 INFO sqlalchemy.engine.base.Engine (1,)
1 Led zeppelin 1


In [ ]:
session.new

In [ ]:
session.add_all([artist1,artist2])

In [ ]:
session.commit() #세션을 통해 바인딩 된 엔진을 통해서 우리가만든 인스턴스 artist1,2, 를 SQL로만들고 데이터베이스에다가 물리적으로 저장하는 과정까지 해준다 

session 에는 dirty 라는 게있다. 메모리를 관리한다, 데이터베이스와 싱크와 맞아야한다 , 따라서 계속 감시한다 , 메모리의 값들 이 변경과 , 데이터베이스의  따라서 바뀐게있으면 dirty 에 표시가되고 , commit 을 해야한다는것을 사용자 에게 알려준다 . 

In [ ]:
session.dirty

In [ ]:
session.new

In [ ]:
artist1.name = "xyz"

In [ ]:
session.dirty #바꼈는데 반영되지않는값이 있다고 알려준다

In [ ]:
session.commit() # 따라서 커밋한다

In [ ]:
session.dirty

In [ ]:
session.add_all([
    Album(title="Hysteria",artist_id=1),
    Album(title="21st Century BreakDown",artist_id=2),
    
])

In [ ]:
session.dirty

In [ ]:
session.new #new 에 있다. 값이 변경된것이아니라 , 작업이 메모리에만 있으니까

In [ ]:
session.add_all([
    Genre(name="Rock"),
    Genre(name="Metal"),
])


In [ ]:
session.add_all([
    Track(title="1-1",album_id=1,genre_id=1),
    Track(title="1-2",album_id=1,genre_id=2),
    Track(title="2-1",album_id=2,genre_id=1),
    Track(title="2-2",album_id=2,genre_id=2),
    
    
])

In [ ]:
session.commit()

In [ ]:
temp = Track(title="5-1",album_id=5,genre_id=5)

In [ ]:
type(temp)

In [ ]:
session.add(temp)

In [ ]:
temp.id

In [ ]:
session.new

In [ ]:
for row in session.query(Artist):
    print(row)

In [ ]:
for row in session.query(Artist).filter(Artist.id==1): # 필터를 통해서 where 처럼해서 메모리에 있는것을 불러온다.
    print(row) 

filter 와  filter_by 가 있는데 해당 테이블에 있느냐 없느냐로 해석됨. 

foreign 키를 설정해서 , 클래스르 161부터 목요일 부터  - 장 부터 한줄이면된다 . 릴레이션쉽을 사용하면  목요일날하겠다 